In [7]:
import sys
from os import listdir
import numpy as np 
from xml.dom.minidom import parse
from nltk.tokenize import word_tokenize
from nltk.corpus import stopwords
stopwords =  set(stopwords.words('english'))
import pandas as pd
output_path_name="task9.2_raquel_5.txt"

output_path =  "evaluations/" + output_path_name
results_path = output_path.replace('.txt', '_All_scores.log')
datadir = '../../data/Test-DDI/DrugBank'
training_data = '/home/raquel/Documents/mai/ahlt/data/Train/All'

In [2]:
interaction_words = ['interacts', 'enhances', 'inhibits']
def check_interaction(tokens, entities, e1, e2):
    # Complete this function to return a pair (boolean, ddi_type)
    # depending on whether there is an interaction between e1 and e2
    token_list = [t[0] for t in tokens]
    # print(token_list,'\n', entities,'\n', e1,'\n', e2)
    
    if 'increase' in token_list or 'increased' in token_list: 
        return True, 'mechanism'
    
    if 'effect' in token_list or 'effects' in token_list: 
        return True, 'effect'
    
    if 'patients' and 'may' in token_list:
        return True, 'advise'
    
    if 'may' in token_list:
        return True, 'mechanism'
    return False, "null"

def tokenize(txt):
    offset = 0
    tks = []
    for t in word_tokenize(txt):
        offset = txt.find(t, offset)
        tks.append((t, offset, offset + len(t) - 1))
        offset += len(t)
    return tks


#datadir = sys.argv[1]

def predict(datadir,output_path,test=False):
    # process each file in directory
    with open(output_path, 'w') as file:
        for f in listdir(datadir):

            # parse XML file, obtaining a DOM tree
            tree = parse(datadir + "/" + f)

            # process each sentence in the file
            sentences = tree.getElementsByTagName("sentence")
            for s in sentences:
                sid = s.attributes["id"].value  # get sentence id
                stext = s.attributes["text"].value  # get sentence text

                tokens = tokenize(stext)

                # load sentence entities
                entities = {}
                ents = s.getElementsByTagName("entity")
                for e in ents:
                    id = e.attributes["id"].value
                    offs = e.attributes["charOffset"].value.split("-")
                    entities[id] = offs

                # for each pair in the sentence, decide whether it is DDI and its type
                pairs = s.getElementsByTagName("pair")
                for p in pairs:
                    id_e1 = p.attributes["e1"].value
                    id_e2 = p.attributes["e2"].value
                    (is_ddi, ddi_type) = check_interaction(tokens, entities, id_e1, id_e2)                    
                    ddi = "1" if is_ddi else "0"
                    file.write(sid + "|" + id_e1 + "|" + id_e2 + "|" + ddi + "|" + ddi_type)
                    file.write('\n')
                    if test:
                        return

def show_results(results_path):
    import subprocess
    subprocess.call(['java', '-jar', '../../eval/evaluateDDI.jar', '../../data/Test-DDI/DrugBank', output_path])
    results_file = open(results_path,'r')
    print(results_file.read())
    results_file.close()

predict(datadir,output_path,False)
show_results(results_path)

SCORES FOR THE GROUP: raquel RUN=5
Gold Dataset: /DrugBank

Partial Evaluation: only detection of DDI (regadless to the type)
tp	fp	fn	total	prec	recall	F1
533	2090	351	884	0.2032	0.6029	0.304


Detection and Classification of DDI
tp	fp	fn	total	prec	recall	F1
212	2411	672	884	0.0808	0.2398	0.1209


________________________________________________________________________

SCORES FOR DDI TYPE
Scores for ddi with type mechanism
tp	fp	fn	total	prec	recall	F1
77	776	201	278	0.0903	0.277	0.1362


Scores for ddi with type effect
tp	fp	fn	total	prec	recall	F1
113	645	185	298	0.1491	0.3792	0.214


Scores for ddi with type advise
tp	fp	fn	total	prec	recall	F1
22	990	192	214	0.0217	0.1028	0.0359


Scores for ddi with type int
tp	fp	fn	total	prec	recall	F1
0	0	94	94	0	0	0


MACRO-AVERAGE MEASURES:
	P	R	F1
	0.0653	0.1897	0.0971
________________________________________________________________________




In [3]:
save_info = {}
save_count = {}
def dump_data(datadir):

    # process each file in directory
    for f in listdir(datadir) :
        # parse XML file, obtaining a DOM tree
        tree = parse(datadir+"/"+f)
        # process each sentence in the file
        sentences = tree.getElementsByTagName("sentence")
        for s in sentences :
            sid = s.attributes["id"].value
             # get sentence id
            stext = s.attributes["text"].value # get sentence text
            tokens = tokenize(stext)
            # load and store sentence entities
            entities = {}
            ents = s.getElementsByTagName("entity")
            for e in ents :
                id = e.attributes["id"].value
                offs = e.attributes["charOffset"].value.split("-")
                entities[id] = offs
            # for each pair in the sentence, extract entities
            # and words in between
            pairs = s.getElementsByTagName("pair")
            for p in pairs:
                id_e1 = p.attributes["e1"].value
                id_e2 = p.attributes["e2"].value
                ddi = p.attributes["ddi"].value
                if ddi=="true" and "type" in p.attributes:
                    
                    e1_start = int(entities[id_e1][0])
                    e2_end = int(entities[id_e2][-1])
                    print (p.attributes["type"].value+":", end="")
                    name = p.attributes["type"].value
                    for t in tokens :
                        if t[0] in stopwords:
                            continue
                        if t[0].isalpha():
                            try:
                                save_info[name].append(t[0])
                                save_count[name] +=1
                            except:
                                save_info[name] = []
                                save_count[name] = 0
                        if (t[1]>=e1_start and t[2]<=e2_end):
                            print (" "+t[0], end="")
                    print("")

dump_data(datadir)

mechanism: Aminoglutethimide administered concomitantly depo-subQ provera 104
effect: ProAmatine , cardiac glycosides
effect: phenylephrine , pseudoephedrine , ephedrine , phenylpropanolamine dihydroergotamine ) may enhance potentiate pressor effects ProAmatine
effect: pseudoephedrine , ephedrine , phenylpropanolamine dihydroergotamine ) may enhance potentiate pressor effects ProAmatine
effect: ephedrine , phenylpropanolamine dihydroergotamine ) may enhance potentiate pressor effects ProAmatine
effect: phenylpropanolamine dihydroergotamine ) may enhance potentiate pressor effects ProAmatine
effect: dihydroergotamine ) may enhance potentiate pressor effects ProAmatine
effect: Alpha-adrenergic blocking agents , prazosin , terazosin , doxazosin , antagonize effects ProAmatine
effect: prazosin , terazosin , doxazosin , antagonize effects ProAmatine
effect: terazosin , doxazosin , antagonize effects ProAmatine
effect: doxazosin , antagonize effects ProAmatine
mechanism: desglymidodrine ( ba

effect: Anticholinergics antagonize effects antiglaucoma agents
effect: Anticholinergic drugs presence increased intraocular pressure may hazardous taken concurrently agents corticosteroids
mechanism: Anticholinergic agents may affect gastrointestinal absorption various drugs , slowly dissolving dosage forms digoxin
effect: Anticholinergic drugs may antagonize effects drugs alter gastrointestinal motility , metoclopramide
mechanism: antacids may interfere absorption anticholinergic agents
int: Trilostane may interact aminoglutethimide
int: Trilostane may interact aminoglutethimide mitotane
advise: Matulane exhibits monoamine oxidase inhibitory activity , sympathomimetic drugs
advise: Matulane exhibits monoamine oxidase inhibitory activity , sympathomimetic drugs , tricyclic antidepressant
advise: Matulane exhibits monoamine oxidase inhibitory activity , sympathomimetic drugs , tricyclic antidepressant drugs ( e.g. , amitriptyline HCl
advise: Matulane exhibits monoamine oxidase inhibito

int: Methscopolamine may interact antidepressants ( tricyclic type ) , MAO inhibitors ( e.g. , phenelzine , linezolid , tranylcypromine , isocarboxazid , selegiline , furazolidone ) , quinidine , amantadine , antihistamines ( e.g. , diphenhydramine ) , anticholinergics , potassium chloride supplements , antacids , absorbent-type anti-diarrhea medicines ( e.g. , kaolin-pectin ) , phenothiazines
int: Methscopolamine may interact antidepressants ( tricyclic type ) , MAO inhibitors ( e.g. , phenelzine , linezolid , tranylcypromine , isocarboxazid , selegiline , furazolidone ) , quinidine , amantadine , antihistamines ( e.g. , diphenhydramine ) , anticholinergics , potassium chloride supplements , antacids , absorbent-type anti-diarrhea medicines ( e.g. , kaolin-pectin ) , phenothiazines ( e.g. , chlorpromazine
int: Methscopolamine may interact antidepressants ( tricyclic type ) , MAO inhibitors ( e.g. , phenelzine , linezolid , tranylcypromine , isocarboxazid , selegiline , furazolidone ) 

mechanism: pantoprazole may interfere absorption drugs gastric pH important determinant bioavailability ( eg , ketoconazole , ampicillin
mechanism: pantoprazole may interfere absorption drugs gastric pH important determinant bioavailability ( eg , ketoconazole , ampicillin esters , iron
effect: NSAIDs may diminish antihypertensive effect angiotensin-converting enzyme ( ACE ) inhibitors
advise: NSAIDs concomitantly ACE inhibitors
mechanism: aspirin ( 1000 mg TID ) healthy volunteers tended increase AUC ( 10 % ) Cmax ( 24 % ) meloxicam
advise: NSAIDs , concomitant administration meloxicam aspirin
advise: meloxicam aspirin
effect: aspirin MOBIC
mechanism: cholestyramine significantly increased clearance meloxicam
effect: NSAIDs reduce natriuretic effect furosemide
effect: NSAIDs reduce natriuretic effect furosemide thiazide diuretics
advise: furosemide MOBIC
mechanism: NSAIDs produced elevation plasma lithium
mechanism: lithium doses ranging 804 1072 mg BID meloxicam
advise: lithium treat

advise: Scopolamine used care patients taking drugs capable causing CNS effects sedatives
advise: Scopolamine used care patients taking drugs capable causing CNS effects sedatives , tranquilizers
advise: Scopolamine used care patients taking drugs capable causing CNS effects sedatives , tranquilizers , alcohol
effect: ASPIRIN AND OTHER SALICYLATE DRUGS WILL BE ADDITIVE TO DISALCID
effect: SALICYLATE DRUGS WILL BE ADDITIVE TO DISALCID
effect: Salicylates given concomitantly anticoagulant drugs
effect: Salicylates may enhance hypoglycemic effect oral antidiabetic drugs
mechanism: Salicylate competes number drugs protein binding sites , notably penicillin
mechanism: Salicylate competes number drugs protein binding sites , notably penicillin , thiopental
mechanism: Salicylate competes number drugs protein binding sites , notably penicillin , thiopental , thyroxine
mechanism: Salicylate competes number drugs protein binding sites , notably penicillin , thiopental , thyroxine , triiodothyron

In [4]:
from collections import Counter
print(save_count)
for k in save_info:
    print(k, '\n',Counter(save_info[k]).most_common()[:20])

{'mechanism': 4974, 'effect': 5014, 'advise': 3743, 'int': 2423}
mechanism 
 [('may', 115), ('drugs', 93), ('clearance', 82), ('phenytoin', 55), ('increase', 52), ('plasma', 51), ('metabolism', 51), ('concentrations', 50), ('levels', 49), ('ketoconazole', 42), ('secretion', 41), ('system', 39), ('AUC', 37), ('phenobarbital', 35), ('oral', 35), ('pharmacokinetic', 34), ('administered', 33), ('increased', 32), ('renal', 29), ('itraconazole', 29)]
effect 
 [('may', 180), ('effects', 106), ('drugs', 100), ('agents', 93), ('The', 71), ('inhibitors', 65), ('patients', 60), ('increase', 56), ('effect', 42), ('administered', 40), ('concomitantly', 39), ('use', 38), ('anticholinergic', 37), ('alcohol', 35), ('bleeding', 31), ('reported', 29), ('activity', 29), ('blocking', 28), ('meperidine', 28), ('CNS', 26)]
advise 
 [('patients', 63), ('drugs', 60), ('ketoconazole', 49), ('may', 44), ('recommended', 41), ('administered', 38), ('receiving', 36), ('inhibitors', 36), ('dose', 33), ('paricalcito

In [ ]:
train_df = pd.read_csv(train_df_path, index_col=0)
from sklearn.feature_extraction.text import CountVectorizer


sentences = train_df.sentence_text.values
y = train_df['relation_type'].values

from sklearn.model_selection import train_test_split

sentences_train, sentences_val, y_train, y_val = train_test_split(sentences, y, test_size=0.25, random_state=1000, stratify=y)

In [ ]:
from sklearn.linear_model import LogisticRegression

classifier = LogisticRegression()
classifier.fit(X_train, y_train)
y_pred = classifier.predict(X_val)

df_confusion = pd.crosstab(y_val, y_pred)

score = classifier.score(X_val, y_val)
print("Accuracy:", score)
df_confusion

In [9]:
 train_df = pd.read_csv(train_df_path, index_col=0)

NameError: name 'train_df_path' is not defined

In [8]:
def pre_processing(sentences_train):
    pass

SyntaxError: unexpected EOF while parsing (<ipython-input-8-6040ebbe3c13>, line 2)

In [116]:
def train_baseline():
    train_df = pd.read_csv(train_df_path, index_col=0)
    sentences_train = train_df.sentence_text.values
    
    y_train = train_df['relation_type'].values
    print(Counter(y_train))
    vectorizer = CountVectorizer()
    vectorizer.fit(sentences_train)
    X_train = vectorizer.transform(sentences_train)
    classifier = LogisticRegression()
    classifier.fit(X_train, y_train)
    return vectorizer, classifier

vectorizer, classifier = train_baseline()

Counter({'none': 23771, 'effect': 1687, 'mechanism': 1319, 'advise': 826, 'int': 188})


In [121]:
def check_interaction(tokens, entities, e1, e2):
    # uses the vectorizer and the classifier already trained
    sentence_array = vectorizer.transform([tokens])
    y_pred = classifier.predict(sentence_array)

    if y_pred[0] == 'none':
        return False, "null"
    else:
        return True, y_pred[0]

def predict(datadir,output_path,test=False):
    # process each file in directory
    with open(output_path, 'w') as file:
        for f in listdir(datadir):

            # parse XML file, obtaining a DOM tree
            tree = parse(datadir + "/" + f)

            # process each sentence in the file
            sentences = tree.getElementsByTagName("sentence")
            for s in sentences:
                sid = s.attributes["id"].value  # get sentence id
                stext = s.attributes["text"].value  # get sentence text

                # load sentence entities
                entities = {}
                ents = s.getElementsByTagName("entity")
                for e in ents:
                    id = e.attributes["id"].value
                    offs = e.attributes["charOffset"].value.split("-")
                    entities[id] = offs

                # for each pair in the sentence, decide whether it is DDI and its type
                pairs = s.getElementsByTagName("pair")
                for p in pairs:
                    id_e1 = p.attributes["e1"].value
                    id_e2 = p.attributes["e2"].value
                    (is_ddi, ddi_type) = check_interaction(stext, entities, id_e1, id_e2)                    
                    ddi = "1" if is_ddi else "0"
                    file.write(sid + "|" + id_e1 + "|" + id_e2 + "|" + ddi + "|" + ddi_type)
                    file.write('\n')
                    if test:
                        return

def show_results(results_path):
    import subprocess
    subprocess.call(['java', '-jar', '../../eval/evaluateDDI.jar', '../../data/Test-DDI/All', output_path])
    results_file = open(results_path,'r')
    print(results_file.read())
    results_file.close()

predict(datadir,output_path,False)
show_results(results_path)

SCORES FOR THE GROUP: raquel RUN=5
Gold Dataset: /All

Partial Evaluation: only detection of DDI (regadless to the type)
tp	fp	fn	total	prec	recall	F1
241	528	643	884	0.3134	0.2726	0.2916


Detection and Classification of DDI
tp	fp	fn	total	prec	recall	F1
212	557	672	884	0.2757	0.2398	0.2565


________________________________________________________________________

SCORES FOR DDI TYPE
Scores for ddi with type mechanism
tp	fp	fn	total	prec	recall	F1
42	132	260	302	0.2414	0.1391	0.1765


Scores for ddi with type effect
tp	fp	fn	total	prec	recall	F1
100	337	260	360	0.2288	0.2778	0.2509


Scores for ddi with type advise
tp	fp	fn	total	prec	recall	F1
70	88	151	221	0.443	0.3167	0.3694


Scores for ddi with type int
tp	fp	fn	total	prec	recall	F1
0	0	96	96	0	0	0


MACRO-AVERAGE MEASURES:
	P	R	F1
	0.2283	0.1834	0.2034
________________________________________________________________________


